In [19]:
import pandas as pd
import csv
import pickle
from tqdm import tqdm_notebook as tqdm
import collections
import numpy as np

In [3]:
path = '/home/User1/data/self_citations/'

In [11]:
articles = {}
with open(path+'article.txt','r') as f:
    reader = csv.reader(f,delimiter='\t')
    print(next(reader))
    for line in reader:
        articles[int(line[0])] = int(line[2])

['ID_Art', 'ItemID', 'Annee_Bibliographique', 'Code_Revue', 'Nb_Reference', 'Nb_Auteur']


In [9]:
with open(path+'dict_citant.p','rb') as f:
    dict_citant = pickle.load(f)

In [8]:
with open(path+'dict_cluster_ID.p','rb') as f:
    dict_cluster_ID = pickle.load(f)

In [159]:
authors = list(dict_cluster_ID.keys())
nb_authors = len(authors)
idx = np.arange(nb_authors)
np.random.shuffle(idx)

In [15]:
def find_with_list(myList, target):
    inds = []
    for i in range(len(myList)):
        if myList[i] == target:
            inds += i,
    return inds

In [17]:
def is_in_ordered_list(cites,ordered):
    ordered = set(ordered)
    intersection = cites.intersection(ordered)
    
    if len(intersection) == len(cites):
        in_list = True
    else:
        in_list = False
    return in_list
    

In [227]:
ordered_arts = {}

In [228]:
for i in tqdm(range(nb_authors)):
    cluster_ID = authors[idx[i]]
    year_arts = {}
    arts = np.unique(np.array(dict_cluster_ID[cluster_ID]))
    for art in arts:
        year_arts[art] = articles[art]
    years = np.array(list(year_arts.values())).astype(np.float)
    duplicates = [item for item, count in collections.Counter(years).items() if count > 1]
    #When > 1 art is published in the same year, if A cite B, B comes before
    for dup in duplicates:
        inds = np.array(find_with_list(years,dup))
        arts_dup = arts[inds]
        nb_dup = len(arts_dup)
        citant = {}
        order_arts = []
        for art in arts_dup:
            if art in dict_citant:
                cites = set(dict_citant[art])
                intersection = cites.intersection(set(arts_dup))
            else:
                intersection = set()
            if len(intersection) == 0:
                order_arts.append(art)
            else:
                citant[art] = intersection
        
        key = 0
        while len(citant) > 0:
            art = list(citant.keys())[key]
            cites = citant[art]
            in_list = is_in_ordered_list(cites,order_arts)
            if in_list:
                order_arts.append(art)
                del citant[art]
                key = 0
            else:
                if key == len(citant)-1:
                    break
                else:
                    key += 1
        if len(citant) > 0:
            for art in citant:
                order_arts.append(art)
        years_mod = np.linspace(years[inds[0]],years[inds[0]]+0.99,nb_dup)
        years_map = {}
        for k in range(nb_dup):
            art = order_arts[k]
            id_art = np.where(arts==art)
            years[id_art] = years_mod[k]
    idx_sort = np.argsort(years)
    years_sort = years[idx_sort]
    arts_sort = arts[idx_sort]
    ordered_arts[cluster_ID] = np.concatenate([arts_sort,years_sort],axis=0)
            
            


/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


A Jupyter Widget

In [229]:
with open(path+'ordered_arts.p','wb') as f:
    pickle.dump(ordered_arts,f)

In [210]:
for dup in duplicates:
    inds = np.array(find_with_list(years,dup))
    arts_dup = arts[inds]
    nb_dup = len(arts_dup)
    citantt = {}
    order_arts = []
    for art in arts_dup:
        if art in dict_citant:
            cites = set(dict_citant[art])
            intersection = cites.intersection(set(arts_dup))
        else:
            intersection = set()
        if len(intersection) == 0:
            order_arts.append(art)
        else:
            citantt[art] = intersection
    if len(citantt) > 0:
        print('found')
        break

found


In [212]:
for dup in duplicates:
    inds = np.array(find_with_list(years,dup))
    arts_dup = arts[inds]
    nb_dup = len(arts_dup)
    citant = {}
    order_arts = []
    for art in arts_dup:
        if art in dict_citant:
            cites = set(dict_citant[art])
            intersection = cites.intersection(set(arts_dup))
        else:
            intersection = set()
        if len(intersection) == 0:
            order_arts.append(art)
        else:
            citant[art] = intersection
    key = 0
    while len(citant) > 0:
        art = list(citant.keys())[key]
        cites = citant[art]
        in_list = is_in_ordered_list(cites,order_arts)
        if in_list:
            order_arts.append(art)
            del citant[art]
            key = 0
        else:
            if key == len(citant)-1:
                break
            else:
                key += 1
    if len(citant) > 0:
        for art in citant:
            order_arts.append(art)
    years_mod = np.linspace(years[inds[0]],years[inds[0]]+0.99,nb_dup)
    years_map = {}
    for k in range(nb_dup):
        art = order_arts[k]
        id_art = np.where(arts==art)
        years[id_art] = years_mod[k]
    


In [213]:
idx_sort = np.argsort(years)
years_sort = years[idx_sort]
arts_sort = arts[idx_sort]

In [211]:
citantt

{50549820: {49957664}, 51565941: {49957664, 50549820}}

In [214]:
years_sort

array([2010.   , 2011.   , 2013.   , 2013.495, 2013.99 ])

In [215]:
arts_sort

array([44844472, 46330679, 49957664, 50549820, 51565941])

In [117]:
years

array([2015.  , 2016.  , 2017.  , 2016.  , 2017.33, 2009.  , 2010.  ,
       2011.  , 2011.  , 2010.  , 2013.  , 2013.  , 2014.  , 2015.  ,
       2016.  , 2016.  , 2016.  , 2017.66, 2016.  , 2017.99])

In [48]:
years.dtype

dtype('int64')